In [12]:
import pandas as pd
import numpy as np

In [13]:
df_train = pd.read_csv("data/train.csv")
X = df_train.iloc[:, :-1].values
y = df_train.iloc[:, -1].values

df_kaggle = pd.read_csv("data/test.csv")
X_kaggle = df_kaggle.iloc[:, :].values

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder

# ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(handle_unknown="ignore"), [0, 1])], remainder="passthrough")
# X = np.array(ct.fit_transform(X))
# X_kaggle = np.array(ct.transform(X_kaggle))

In [14]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train = X
y_train = y

In [16]:
list(df_train).index("quartos")

83

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train[:, 84:89] = sc.fit_transform(X_train[:, 84:89])

# sc_y = StandardScaler()
# y_train = sc_y.fit_transform(y_train.reshape(-1,1))

sc_kaggle = StandardScaler()
X_kaggle[:, 83:88] = sc_kaggle.fit_transform(X_kaggle[:, 83:88])

In [ ]:
from catboost import CatBoostRegressor

regressor = CatBoostRegressor()
regressor.fit(X_train, y_train)

In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix

# y_pred = classifier.predict(X_test)
# acc_sc = accuracy_score(y_test, y_pred)
# cf_mx = confusion_matrix(y_test, y_pred)
# print("Accuracy Score:", acc_sc)
# print(cf_mx)

In [ ]:
from sklearn.model_selection import cross_val_score
from datetime import datetime as dt
timestamp = dt.today()

accuracies = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10)
print(f"Accuracy: {accuracies.mean()}")
print(f"StdDev: {accuracies.std()}")
try:
    with open("data/cross_val.log", mode="x+") as cvlog:
        cvlog.write("timestamp,accuracy,stddev")
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")
except:
    with open("data/cross_val.log", mode="a") as cvlog:
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")

In [ ]:
y_kaggle = sc_y.inverse_transform(regressor.predict(X_kaggle))

In [ ]:
with open(f"data/kaggle/kaggle_{timestamp.strftime('%d-%m-%Y_%Hh%M')}.csv", mode="w") as out:
    out.write("Id,preco")
    for i in range(len(y_kaggle)):
        out.write(f"\n{i},{round(y_kaggle[i], 2):.2f}")